# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
import sys
print(sys.path)
%reload_ext dotenv
%dotenv
#%dotenv list

In [1]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

/home/sergey/miniconda3/envs/scale2prod/lib/python3.12/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [2]:
import os
from glob import glob
price_data_path = os.getenv('PRICE_DATA')
price_files_path = os.path.join(price_data_path, '*/*/*.parquet')
price_files = glob(price_files_path)
print(price_files[:3])
# completed repartition as lazy execution ll times fails with: Partition size is less than overlapping window size
data_frames = dd.read_parquet(price_files).repartition(partition_size="1024Mb").set_index('ticker')


['/home/sergey/PycharmProjects/DSI-study/scaling_to_production/05_src/data/prices/ISRG/ISRG_2009.parquet/part.0.parquet', '/home/sergey/PycharmProjects/DSI-study/scaling_to_production/05_src/data/prices/ISRG/ISRG_2017.parquet/part.0.parquet', '/home/sergey/PycharmProjects/DSI-study/scaling_to_production/05_src/data/prices/ISRG/ISRG_2007.parquet/part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [3]:
# Write your code below.
dd_feat = \
    (data_frames.groupby('ticker', group_keys=False).apply(
         lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
    ).assign(
        Adj_Close = lambda x: x['Close']
    ).assign(
        Adj_Close_lag_1 = lambda x: x['Adj_Close'].shift(1)
    ).assign(
        returns = lambda x: x['Close']/x['Close_lag_1'] - 1
    ).assign(
        hi_lo_range = lambda x: (x['High'] - x['Low'])
    ))
dd_feat.compute()

/tmp/ipykernel_71630/3976816096.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  (data_frames.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.613007,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,51.502148,NaN,NaN,8.270744
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.281467,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,47.567955,51.502148,-0.076389,2.950645
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.782806,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,44.617310,47.567955,-0.062030,4.425964
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.344177,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,42.918453,44.617310,-0.038076,2.771820
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.372845,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,46.494991,42.918453,0.083333,4.962444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTS,2024-06-24,171.710007,174.289993,170.300003,171.839996,171.839996,1885900,Health Care,Pharmaceuticals,2024,171.089996,171.839996,171.089996,0.004384,3.989990
ZTS,2024-06-25,171.789993,172.419998,167.720001,167.979996,167.979996,2293400,Health Care,Pharmaceuticals,2024,171.839996,167.979996,171.839996,-0.022463,4.699997
ZTS,2024-06-26,167.210007,171.509995,166.449997,170.899994,170.899994,2053800,Health Care,Pharmaceuticals,2024,167.979996,170.899994,167.979996,0.017383,5.059998


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [4]:
# Write your code below.
# Convert the Dask data frame to a pandas data frame. 
pd_frames = dd_feat.compute()
# Add a rolling average return calculation with a window of 10 days. 
pd_frames['rolling_avg_return'] = pd_frames['returns'].rolling(window=10).mean()
print(pd_frames.tail())

             Date        Open        High         Low       Close   Adj Close  \
ticker                                                                          
ZTS    2024-06-24  171.710007  174.289993  170.300003  171.839996  171.839996   
ZTS    2024-06-25  171.789993  172.419998  167.720001  167.979996  167.979996   
ZTS    2024-06-26  167.210007  171.509995  166.449997  170.899994  170.899994   
ZTS    2024-06-27  174.490005  181.660004  174.259995  176.460007  176.460007   
ZTS    2024-06-28  176.460007  177.070007  173.029999  173.360001  173.360001   

         Volume       sector        subsector  year  Close_lag_1   Adj_Close  \
ticker                                                                         
ZTS     1885900  Health Care  Pharmaceuticals  2024   171.089996  171.839996   
ZTS     2293400  Health Care  Pharmaceuticals  2024   171.839996  167.979996   
ZTS     2053800  Health Care  Pharmaceuticals  2024   167.979996  170.899994   
ZTS     4164700  Health Care  Ph

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ No was not necessary to convert to a pandas DataFrame to calculate the moving average return, as Dask can handle rolling operations directly on its frames
+ Would it have been better to do it in Dask? Why?
+ It is better to use Dask as it can carry in parallel large datasets. It also supports to panda format conversion.
(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.